In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import sys; sys.argv=['']; del sys

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.batchN1= nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.batchN2= nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.batchN3 = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.batchN1(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.batchN2(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.batchN3(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [18]:
class Net_BatchNorm(nn.Module):
    def __init__(self):
        super(Net_BatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
class Net_sigmoid(nn.Module):
    def __init__(self):
        super(Net_sigmoid, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = torch.sigmoid(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
class Net_dropOut(nn.Module):
    def __init__(self,dropout=0.5):
        super(Net_dropOut, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [7]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
def InitialModel(train_loader,test_loader,device, args):
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [9]:
def Model_Sigmoid(train_loader,test_loader,device, args):
    model = Net_sigmoid().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [16]:
def Model_DropOut(train_loader,test_loader,device, args):
    for dropout in [0.25,0.5,0.75,1]:
        print("\n\n DropOut is added for dropout value::",dropout,"\n")

        model = Net_dropOut(dropout).to(device)
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

        for epoch in range(1, args.epochs + 1):
            train(args, model, device, train_loader, optimizer, epoch)
            test(args, model, device, test_loader)

In [19]:
def Model_BatchNorm(train_loader,test_loader,device, args):
    model = Net_BatchNorm().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

In [11]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 10000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=1000, metavar='N',
                    help='how many batches to wait before logging training status')

parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')
args = parser.parse_args()
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


    

In [12]:
print("For ReLU model:")
InitialModel(train_loader,test_loader,device, args)

For ReLU model:
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039

Test set: Average loss: 0.1020, Accuracy: 9662/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.145645

Test set: Average loss: 0.0613, Accuracy: 9826/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.055441

Test set: Average loss: 0.0564, Accuracy: 9804/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.019864

Test set: Average loss: 0.0409, Accuracy: 9860/10000 (99%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.010258

Test set: Average loss: 0.0382, Accuracy: 9870/10000 (99%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.132001

Test set: Average loss: 0.0333, Accuracy: 9896/10000 (99%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.029959

Test set: Average loss: 0.0345, Accuracy: 9874/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.004609

Test set: Average loss: 0.0390, Accuracy: 9875/10000 (99%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.081802

Test set: Average loss: 0.0288, Accuracy: 9909/10000 (99%)

Train Epoch: 10 [0/60000 (0%)]

In [13]:
print("For Sigmoid model:")
Model_Sigmoid(train_loader,test_loader,device, args)

For Sigmoid model:
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.338853

Test set: Average loss: 2.3136, Accuracy: 1843/10000 (18%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.281903

Test set: Average loss: 2.2830, Accuracy: 1095/10000 (11%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.273887

Test set: Average loss: 2.0850, Accuracy: 3225/10000 (32%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.108958

Test set: Average loss: 0.9830, Accuracy: 7534/10000 (75%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.895201

Test set: Average loss: 0.5749, Accuracy: 8312/10000 (83%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.636260

Test set: Average loss: 0.4403, Accuracy: 8690/10000 (87%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.462399

Test set: Average loss: 0.3647, Accuracy: 8925/10000 (89%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.253726

Test set: Average loss: 0.3106, Accuracy: 9085/10000 (91%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.405763

Test set: Average loss: 0.2711, Accuracy: 9185/10000 (92%)

Train Epoch: 10 [0/60000 (0

In [17]:
print("Dropout added to the model::::\n\n")
args.epochs = 5
Model_DropOut(train_loader,test_loader,device, args)

Dropout added to the model::::


DropOut is added for dropout value:: 0.25 

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.310330

Test set: Average loss: 0.1175, Accuracy: 9763/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.202403

Test set: Average loss: 0.0763, Accuracy: 9828/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.015938

Test set: Average loss: 0.0674, Accuracy: 9853/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.066971

Test set: Average loss: 0.0516, Accuracy: 9871/10000 (99%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.021871

Test set: Average loss: 0.0471, Accuracy: 9888/10000 (99%)

DropOut is added for dropout value:: 0.5 

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.350428

Test set: Average loss: 0.2569, Accuracy: 9772/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.082496

Test set: Average loss: 0.1735, Accuracy: 9831/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.024754

Test set: Average loss: 0.1606, Accuracy: 9864/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Lo

In [ ]:
print("BatchNorm with RELU")
Model_BatchNorm(train_loader,test_loader,device, args)

BatchNorm with RELU
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309061
